In [1]:
#PART1 - Converting WIKI table into PANDA DATAFRAME
# Import libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup 
# Webscrapping the URL
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(url)
print(page.status_code)
soup = BeautifulSoup(page.text,"html.parser")

200


In [2]:
# READ Table
Table_array = []
Table_text_element = soup.find_all( class_= "wikitable sortable")
Table_text_element=Table_text_element[0]
for row in Table_text_element.find_all('tr'):
        for cell in row.find_all('td'):
             Table_array.append(cell.text)
Table_array1string=', '.join(Table_array)
Table_array1string = 'Column,PostCode,Borough,Neighbourhood' + '\n, '+ Table_array1string
fh = open ('Testfile.txt', 'w',)
fh.write(Table_array1string)
fh.close()
df = pd.read_csv('Testfile.txt')
#Remove the unwanted Column as part of Cleansing
df.drop('Column',axis=1,inplace=True)

In [3]:
# q1 Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
# Creating new dataframe excluding column - Borough having value 
df_q1= df[df['Borough'] != ' Not assigned']
print(df_q1.shape)
df_q1.head()

(211, 3)


,PostCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [4]:
#Q3 If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 
#Replacing Neighbourhood value with borough if its ' Not assigned'
df_q1.loc[df_q1['Neighbourhood'] == ' Not assigned', 'Neighbourhood'] = df_q1.Borough
print (df_q1.shape)

#Q2 More than one neighborhood can exist in one postal code area. 
#For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: 
#Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as 
#shown in row 11 in the above table.

res= df_q1.groupby(['PostCode','Borough']).Neighbourhood.agg([('Neighbourhood', ', '.join)]).reset_index()
print("Number of rows in the final dataframe as per .Shape ", res.shape)

res.columns = ['Postal Code', 'Borough', 'Neighbourhood']
res.head() 
res.tail()
# END OF PART1

(211, 3)
Number of rows in the final dataframe as per .Shape  (103, 3)


/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,Postal Code,Borough,Neighbourhood
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Ri..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergat..."
102,M9W,Etobicoke,Northwest


In [5]:
# PART 2 - MERGING IT WITH GEOSPATIAL DATA BEING PROVIDED
# Reading it from the geo file -http://cocl.us/Geospatial_data
print(type(res))
url = "http://cocl.us/Geospatial_data"
geodf = pd.read_csv(url)
geodf.columns = ['GeoPostCode','Latitude', 'Longitude']


<class 'pandas.core.frame.DataFrame'>


In [6]:
# Quick verification & Comparison with the earlier data frame  res in Part1 , before we merge it.
print(geodf.head())
print(geodf.tail())

  GeoPostCode   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476
    GeoPostCode   Latitude  Longitude
98          M9N  43.706876 -79.518188
99          M9P  43.696319 -79.532242
100         M9R  43.688905 -79.554724
101         M9V  43.739416 -79.588437
102         M9W  43.706748 -79.594054


In [7]:
Concatdf = pd.concat([res,geodf], axis=1)
finalgeodf=Concatdf.drop(['GeoPostCode'] , axis=1)
finalgeodf.head()
# End of Part2

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [8]:
# PART 3 - Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto 

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [10]:
address = 'Toronto, CA'
geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [29]:
# Extracting those boroughs which got Toronto in it from the original data frame finalgeodf
finalgeodf.head()
Torontogeodf = finalgeodf.loc[finalgeodf['Borough'].str.contains('Toronto')].reset_index()
Torontogeodf.shape
Torontogeodf

,index,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,37,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,43,M4M,East Toronto,Studio District,43.659526,-79.340923
4,44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,45,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,47,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,48,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
9,49,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, Sout...",43.686412,-79.400049


In [19]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Torontogeodf['Latitude'], Torontogeodf['Longitude'], Torontogeodf['Borough'], Torontogeodf['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

In [13]:
# Exploring Venues using FourSquare
CLIENT_ID = 'AV2RXHWVXPVA2W4UAFKRNVMEINKR3U2RAQYF2XBVARV3U0PG' # your Foursquare ID
CLIENT_SECRET = 'PJIWCB43CE1JXMLIXKSCVWNOQ2HM130004DB0KOQX5MHHXEB' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: AV2RXHWVXPVA2W4UAFKRNVMEINKR3U2RAQYF2XBVARV3U0PG
CLIENT_SECRET:PJIWCB43CE1JXMLIXKSCVWNOQ2HM130004DB0KOQX5MHHXEB


In [32]:
# Exploring the Venues nearby the Neighbourhood - Rosedale in Downtown TOronto
Torontogeodf.loc[10,'Neighbourhood']

' Rosedale'

In [33]:
neighborhood_latitude = Torontogeodf.loc[10, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Torontogeodf.loc[10, 'Longitude'] # neighborhood longitude value

neighborhood_name = Torontogeodf.loc[10, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of  Rosedale are 43.6795626, -79.37752940000001.


In [34]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=AV2RXHWVXPVA2W4UAFKRNVMEINKR3U2RAQYF2XBVARV3U0PG&client_secret=PJIWCB43CE1JXMLIXKSCVWNOQ2HM130004DB0KOQX5MHHXEB&v=20180605&ll=43.6795626,-79.37752940000001&radius=500&limit=100'

In [26]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d7bdedccc7d41002b26e304'},
 'response': {'headerLocation': 'Rosedale',
  'headerFullLocation': 'Rosedale, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 5,
  'suggestedBounds': {'ne': {'lat': 43.6840626045, 'lng': -79.37131878274371},
   'sw': {'lat': 43.675062595499995, 'lng': -79.38374001725632}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bae2150f964a520df873be3',
       'name': 'Mooredale House',
       'location': {'address': '146 Crescent Rd.',
        'crossStreet': 'btwn. Lamport Ave. and Mt. Pleasant Rd.',
        'lat': 43.678630645646535,
        'lng': -79.38009142511322,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.678630645646535,
          'lng': -79.380091425113

In [27]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [39]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Mooredale House,Building,43.678631,-79.380091
1,Rosedale Park,Playground,43.682328,-79.378934
2,Whitney Park,Park,43.682036,-79.373788
3,Alex Murray Parkette,Park,43.678300,-79.382773
4,Milkman's Lane,Trail,43.676352,-79.373842


In [38]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

5 venues were returned by Foursquare.


In [40]:
# Creating a function to repeat this for all neighbourhood across downtown Toronto
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [41]:
DownTownToronto_venues = getNearbyVenues(names=Torontogeodf['Neighbourhood'],
                                   latitudes=Torontogeodf['Latitude'],
                                   longitudes=Torontogeodf['Longitude']
                                  )

 The Beaches
 The Danforth West,  Riverdale
 The Beaches West,  India Bazaar
 Studio District
 Lawrence Park
 Davisville North
 North Toronto West
 Davisville
 Moore Park,  Summerhill East
 Deer Park,  Forest Hill SE,  Rathnelly,  South Hill,  Summerhill West
 Rosedale
 Cabbagetown,  St. James Town
 Church and Wellesley
 Harbourfront,  Regent Park
 Ryerson,  Garden District
 St. James Town
 Berczy Park
 Central Bay Street
 Adelaide,  King,  Richmond
 Harbourfront East,  Toronto Islands,  Union Station
 Design Exchange,  Toronto Dominion Centre
 Commerce Court,  Victoria Hotel
 Roselawn
 Forest Hill North,  Forest Hill West
 The Annex,  North Midtown,  Yorkville
 Harbord,  University of Toronto
 Chinatown,  Grange Park,  Kensington Market
 CN Tower,  Bathurst Quay,  Island airport,  Harbourfront West,  King and Spadina,  Railway Lands,  South Niagara
 Stn A PO Boxes 25 The Esplanade
 First Canadian Place,  Underground city
 Christie
 Dovercourt Village,  Dufferin
 Little Portugal,  Trin

In [44]:
print(DownTownToronto_venues.shape)
DownTownToronto_venues.head()

(1708, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant
